# Example Network

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Generate synthetic dataset
X, y = make_moons(n_samples=1000, noise=0.2, random_state=0)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Convert to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Define simple NN
class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.net(x)

model = SimpleNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(300):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

# Evaluation
model.eval()
with torch.no_grad():
    preds = (model(X_test) > 0.5).float()
    accuracy = (preds.eq(y_test).sum() / len(y_test)).item()
print(f'Test Accuracy: {accuracy:.4f}')


Test Accuracy: 0.9633


# Define Attack Setup

In [4]:
from framework.attack import Injector
from framework.criterion import classification_accuracy
import pandas as pd

inj = Injector(model,X = X_test,y = y_test, criterion=classification_accuracy)

results = inj.run_seu(0)

display(pd.DataFrame(results))

results = inj.run_stochastic_seu(0,0.3)

display(pd.DataFrame(results))

Testing a forward pass on cuda...
Basline Criterion Score: 0.9633333333333334
Testing Layer:  net.0.weight


16it [00:00, 548.08it/s]


Testing Layer:  net.0.bias


8it [00:00, 404.20it/s]


Testing Layer:  net.2.weight


...

Testing Layer:  net.2.weight


8it [00:00, 503.72it/s]


Testing Layer:  net.2.bias


1it [00:00, ?it/s]


,tensor_location,criterion_score,layer_name,value_before,value_after
0,"(0, 1)",0.970000,net.0.weight,-0.932331,0.932331
1,"(4, 0)",0.873333,net.0.weight,1.873351,-1.873351
2,"(4, 1)",0.960000,net.0.weight,0.983437,-0.983437
3,"(0,)",0.933333,net.0.bias,-0.866254,0.866254
4,"(3,)",0.806667,net.0.bias,-1.862231,1.862231
5,"(0, 2)",0.946667,net.2.weight,-0.969910,0.969910
6,"(0, 5)",0.666667,net.2.weight,1.238899,-1.238899
7,"(0, 6)",0.963333,net.2.weight,-0.101456,0.101456
8,"(0, 7)",0.760000,net.2.weight,-2.798505,2.798505
9,"(0,)",0.963333,net.2.bias,-0.295556,0.295556
